code by Huan Li

In [ ]:
import pytesseract
from pathlib import Path
import fitz  # PyMuPDF
import cv2
import numpy as np
import pandas as pd

#  Tesseract executable path
pytesseract.pytesseract.tesseract_cmd = r"C:\Program Files\Tesseract-OCR\tesseract.exe"

In [77]:
# project root

BASE_DIR = Path.cwd()

SPEECHES_DIR = BASE_DIR / "data" / "speeches"
PDF_DIR = SPEECHES_DIR / "pdf"
PDF_SINGLE_COLUMN_DIR = SPEECHES_DIR / "pdf_single_column"
TXT_OCR_DIR = SPEECHES_DIR / "txt_ocr"
EXTRACTED_OCR_DIR = SPEECHES_DIR / "extracted_ocr"


In [78]:
DATAFRAMES_DIR = BASE_DIR / "data" / "dataframes"
META_PATH = DATAFRAMES_DIR / "metadata_s03.csv"

df = pd.read_csv(META_PATH)

In [ ]:
# extract single column pdf
trial_df = df[(df["text_based"] == False) & (df["two_column_layout"] == False)]

Apply ocr to pdf pages and get text per page


In [ ]:
def ocr_pdf_pages(
    pdf_path,
    dpi=300,
    lang="eng",
    start_page=1,
):
    """
    OCR a PDF into page-level text.

    Parameters
    ----------
    pdf_path : Path or str
    dpi : int
        Rendering DPI
    lang : str
        Tesseract language
    start_page : int
        0-based page index to start OCR (default: 3)

    Returns
    -------
    List[str]
        One OCR text string per page
    """

    doc = fitz.open(pdf_path)
    pages = []

    for i in range(start_page, len(doc)):
        page = doc[i]

        # PDF → image
        pix = page.get_pixmap(dpi=dpi)
        img = np.frombuffer(pix.samples, dtype=np.uint8).reshape(
            pix.height, pix.width, pix.n
        )

        # grayscale
        gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

        # denoise
        blur = cv2.medianBlur(gray, 3)
        opened = cv2.morphologyEx(
            blur, cv2.MORPH_OPEN, np.ones((2, 2), np.uint8)
        )

        # contrast enhancement
        clahe = cv2.createCLAHE(2.0, (8, 8))
        enhanced = clahe.apply(opened)

        # adaptive threshold
        binary = cv2.adaptiveThreshold(
            enhanced, 255,
            cv2.ADAPTIVE_THRESH_GAUSSIAN_C,
            cv2.THRESH_BINARY,
            35, 11
        )

        # OCR
        text = pytesseract.image_to_string(binary, lang=lang)
        pages.append(text)

        print(f"OCR page {i+1}/{len(doc)} done")

    return pages


In [ ]:
import re

# regex patterns for filtering and cleaning
_RE_NOTE_ONLY = re.compile(r"^\d+\s+See\s+Official\s+Records", re.I)
_RE_GA_HEADER_SHORT = re.compile(r"General\s+Assembly", re.I)
_RE_GA_PLENARY_LINE = re.compile(r"General\s+Assembly.*Plenary\s+Meetings", re.I)

# filter some headers / footnotes / noise lines
# EBH/eh A/40/PV.75
# BHS/ow a/40/20.49     (if such appears, add another pattern below)
_RE_DOC_CODE_LINE = re.compile(
    r"""(?ix)
    ^\s*
    (?:[A-Z]{1,5}(?:/[A-Za-z]{1,3})?\s+)?  # filter optional doc code at start    
    (?:A|S|E)/?                   
    (?:C\.\s*\d+/\s*)?                        
    \d+(?:/\d+)*                              
    /PV\.\s*\d+                                
    (?:\s+\d+)?                                
    \s*$
    """
)

_RE_PAGE_RANGE = re.compile(r"^\s*\d+\s*[-–]\s*\d+\s*$")   # 13-15, 9-10
_RE_SINGLE_PAGE = re.compile(r"^\s*\d+\s*$")              # 单独页码行（可选启用）

# filter (The Secretary-General)” type of single parenthesis lines with noise
_RE_SG_PAREN_LINE = re.compile(
    r"(?i)^\s*\(?\s*the\s*[^A-Za-z0-9]{0,3}\s*secretary\s*[^A-Za-z0-9]{0,3}\s*general.*\)?\s*$"
)

def _normalize_sg_markers(s: str) -> str:
    """
    Normalize various weird forms of "Secretary-General" markers in text.
    """
    # normalize quotes and dashes
    s = s.replace("’", "'").replace("‘", "'").replace("`", "'")
    s = s.replace("—", "-").replace("–", "-").replace("−", "-")

    # norm "SECRETARY-~GENERAL" / "SECRETARY–GENERAL" to "SECRETARY-GENERAL"
    s = re.sub(r"(?i)secretary\s*[-~–—]\s*general", "Secretary-General", s)

    # handle "The: SECRETARY-GENERAL" type
    s = re.sub(r"(?i)\bthe\s*:\s*", "The ", s)

    # handle "SECRETARY-GENERAL’" / "SECRETARY-GENERAL;" / "SECRETARY-GENERAL," -> remove trailing abnormal punctuation
    s = re.sub(r"(?i)(Secretary-General)\s*[';，,]+\s*", r"\1 ", s)

    # norm "SECRETARY- GENERAL" (extra spaces in between)
    s = re.sub(r"(?i)secretary\s*-\s*general", "Secretary-General", s)

    # remove weird quotes around SG markers in parentheses (interpretation ‘from Spanish) -> (interpretation from Spanish)
    s = re.sub(r"\(([^)]*)\)", lambda m: "(" + m.group(1).replace("'", "") + ")", s)

    # merge multiple spaces/tabs
    s = re.sub(r"[ \t]+", " ", s)
    return s.strip()


def filter_and_clean_page(page_text: str, min_chars=80):
    """
    Decide whether to keep an OCR page.
    If kept, return cleaned text.
    If dropped, return None.
    """
    text = page_text.strip()

    # --- page-level quick drops ---
    if "." not in text:
        return None
    if len(text) < min_chars:
        return None
    if _RE_NOTE_ONLY.match(text):
        return None
    if _RE_GA_HEADER_SHORT.search(text) and len(text.split()) < 40:
        return None

    cleaned_lines = []

    for line in text.splitlines():
        l = line.strip()
        if not l:
            continue

        # normalize SG markers
        l = _normalize_sg_markers(l)

        # filter headers/footnotes/noise lines
        if _RE_GA_PLENARY_LINE.search(l):
            continue
        if _RE_NOTE_ONLY.match(l):
            continue

        # filter these you screenshot:
        # EBH/eh A/40/PV.75
        # BHS/ow a/40/20.49  (if such appears, add another pattern below)
        if _RE_DOC_CODE_LINE.match(l):
            continue
        if _RE_PAGE_RANGE.match(l):
            continue

        # optional: if you find many "single page number lines" and they affect reading order, enable this
        # if _RE_SINGLE_PAGE.match(l):
        #     continue

        # filter "(The Secretary-General)" type of single parenthesis lines (usually layout noise, not main text)
        if _RE_SG_PAREN_LINE.match(l):
            continue

        cleaned_lines.append(l)

    if not cleaned_lines:
        return None

    return "\n".join(cleaned_lines)


In [82]:
import re

start_sg = re.compile(
    r"(?im)"                  # i: ignorecase, m: multiline
    r"^\s*"
    r"(?:\d+\.?\s*)?"         # optional leading "1." or "12"
    r"(?:The\s*[:;]?\s*)?"    # allow "The" and weird ":" ";" after The
    r"SECRETAR(?:Y|V|C|G)"    # tolerate tiny OCR confusions: Y/V/C/G (可按需要收紧)
    r"[\s\-–—~]*"             # weird dash / ~ between words
    r"GENERAL"                # GENERAL
    r"[’'\)\]\}]*"            # stray closing quotes/brackets after GENERAL
    r"(?:\s*\([^)]*\))?"      # optional "(interpretation from ...)" or similar
    r"\s*[:;]\s*"             # colon OR semicolon as delimiter
)

next_speaker = re.compile(
    r"(?im)"
    r"^\s*"
    r"(?:\d+\.?\s*)?"                # optional numbering
    r"(?:"
        r"The\s*[:;]?\s*"            # "The" with optional weird punct
        r"(?:PRESIDENT|CHAIRMAN|CHAIR|RAPPORTEUR|SECRETARY(?:[\s\-–—~]*GENERAL)?|REPRESENTATIVE|DELEGATE)"
      r"|"
        r"(?:Mr|Mrs|Ms|Miss|Sir|Mme|M\.|Dr|Prof)\.?\s+"  # title
        r"(?:[A-Z][A-Za-z’'\-\.]*\s+){0,5}[A-Z][A-Za-z’'\-\.]*"  # name tokens
    r")"
    r"(?:\s*\([^)]*\)){0,3}"         # optional parentheses blocks
    r"\s*[:;]\s*"                     # colon OR semicolon
)

def extract_sg_from_text(text):
    """
    Extract Secretary-General speech(es) from cleaned document text.

    Parameters
    ----------
    text : str
        Cleaned full document text

    Returns
    -------
    List[str]
        Each element is one SG speech (with header)
    """

    sg_speeches = []
    lines = text.splitlines()

    sg_matches = list(start_sg.finditer(text))

    for sg_match in sg_matches:
        start_idx = sg_match.end()
        start_line = text[:start_idx].count("\n")

        end_line = len(lines)

        # scan forward
        for i in range(start_line + 1, len(lines)):
            if next_speaker.match(lines[i]):
                end_line = i
                break

        speech_body = "\n".join(lines[start_line:end_line]).strip()
        full_speech = sg_match.group().strip() + "\n\n" + speech_body

        sg_speeches.append(full_speech)

    return sg_speeches


In [ ]:
def run_one_pdf(
    pdf_path: Path,
    start_page: int = 1,
    save_intermediate: bool = False
):
    """
    End-to-end pipeline for one UN GA PDF.
    """

    # ---------- Step 0: prepare OCR txt path ----------
    TXT_OCR_DIR.mkdir(parents=True, exist_ok=True)
    ocr_txt_path = TXT_OCR_DIR / f"{pdf_path.stem}.txt"

    # ---------- Step 1: OCR (page-level) ----------
    if ocr_txt_path.exists():
        # apply OCR cache 
        print(f"⏭️ SKIP OCR (exists): {pdf_path.name}")
        full_text = ocr_txt_path.read_text(encoding="utf-8")

    else:
        #  No OCR cache, run OCR
        pages = ocr_pdf_pages(
            pdf_path=pdf_path,
            start_page=start_page
        )

        # ---------- Step 2: filter + clean per page ----------
        clean_pages = []
        for page_text in pages:
            cleaned = filter_and_clean_page(page_text)
            if cleaned:
                clean_pages.append(cleaned)

        # ---------- Step 3: merge ----------
        full_text = "\n\n".join(clean_pages)

        # save OCR result (this is key)
        if save_intermediate:
            ocr_txt_path.write_text(full_text, encoding="utf-8")
            print(f"💾 OCR saved: {ocr_txt_path.name}")

    # ---------- Step 4: extract SG ----------
    sg_speeches = extract_sg_from_text(full_text)

    if sg_speeches:
        EXTRACTED_OCR_DIR.mkdir(parents=True, exist_ok=True)
        out_txt = EXTRACTED_OCR_DIR / f"{pdf_path.stem}.txt"
        out_txt.write_text(sg_speeches[0], encoding="utf-8")
    else:
        # optional: print log for statistics
        print(f"No SG speech found: {pdf_path.name}")

    return sg_speeches

In [ ]:
import time
from pathlib import Path

ok, fail = [], []
t0 = time.time()

DONE_DIR = Path("logs")  
DONE_DIR.mkdir(parents=True, exist_ok=True)

for _, row in trial_df.iterrows():
    pdf_name = row["file_name_pdf"]
    pdf_path = PDF_DIR / pdf_name

    done_flag = DONE_DIR / f"{pdf_path.stem}.done"
    if done_flag.exists():
        print(f"⏭️ SKIP (done): {pdf_name}")
        continue

    try:
        run_one_pdf(
            pdf_path=pdf_path,
            save_intermediate=True
        )

        # mark as done
        done_flag.write_text("ok", encoding="utf-8")

        ok.append(pdf_name)

    except Exception as e:
        fail.append((pdf_name, str(e)))
        print(f"❌ FAIL: {pdf_name}\n   {e}\n")

print("====== 02 OCR + EXTRACTION SUMMARY ======")
print("OK:", len(ok))
print("FAIL:", len(fail))

total_sec = time.time() - t0
print(f"\n⏱️ Total runtime: {total_sec/60:.1f} minutes")


⏭️ SKIP (done): A_1983_C.1_38_PV.11_speeches.pdf
⏭️ SKIP (done): A_1984_C.1_39_PV.12_speeches.pdf
⏭️ SKIP (done): A_1985_40_PV.36_speeches.pdf
⏭️ SKIP (done): A_1985_40_PV.49_speeches.pdf
⏭️ SKIP (done): A_1985_C.1_40_PV.21_speeches.pdf
OCR page 2/56 done
OCR page 3/56 done
OCR page 4/56 done
OCR page 5/56 done
OCR page 6/56 done
OCR page 7/56 done
OCR page 8/56 done
OCR page 9/56 done
OCR page 10/56 done
OCR page 11/56 done
OCR page 12/56 done
OCR page 13/56 done
OCR page 14/56 done
OCR page 15/56 done
OCR page 16/56 done
OCR page 17/56 done
OCR page 18/56 done
OCR page 19/56 done
OCR page 20/56 done
OCR page 21/56 done
OCR page 22/56 done
OCR page 23/56 done
OCR page 24/56 done
OCR page 25/56 done
OCR page 26/56 done
OCR page 27/56 done
OCR page 28/56 done
OCR page 29/56 done
OCR page 30/56 done
OCR page 31/56 done
OCR page 32/56 done
OCR page 33/56 done
OCR page 34/56 done
OCR page 35/56 done
OCR page 36/56 done
OCR page 37/56 done
OCR page 38/56 done
OCR page 39/56 done
OCR page 40